# Import relevant packages

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

/Users/cerenengin/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data 

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label
    
scaled_train_and_validation_data = mnist_train.map(scale)    

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

2024-01-09 16:44:26.777762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-09 16:44:27.496830: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# Model

## Outline the model

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

## Choose the optimizer and the loss function

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [5]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 2s - loss: 0.3386 - accuracy: 0.9047 - val_loss: 0.1769 - val_accuracy: 0.9472 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.1398 - accuracy: 0.9584 - val_loss: 0.1166 - val_accuracy: 0.9632 - 1s/epoch - 3ms/step
Epoch 3/5
540/540 - 1s - loss: 0.0970 - accuracy: 0.9700 - val_loss: 0.0894 - val_accuracy: 0.9728 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.0736 - accuracy: 0.9779 - val_loss: 0.0670 - val_accuracy: 0.9790 - 1s/epoch - 2ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0584 - accuracy: 0.9822 - val_loss: 0.0595 - val_accuracy: 0.9818 - 1s/epoch - 3ms/step


# Test the model

In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 233ms/step - loss: 0.0834 - accuracy: 0.9746


In [7]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.46%
